In [1]:
!pip install "shapely<2.0.0"
!pip install google-cloud-aiplatform>=1.27.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.2 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


In [1]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project="researchproposal", location="us-central1")
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 1
}
model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(
    """You are an investigator looking for code words for drugs hidden in sentences. You have to analyze the sentences and determine whether they contain a code word for drugs. If the sentence contains a code word for drugs, answer as Present: yes and identify the code word in the sentence. If the sentence does not contain any code word for drugs then answer as Present: no.

Q: Lol, that shit is probably so stepped on you can\'t even call it coke anymore.
A: Contains Code word: Yes, Code word : coke, Code word meaning : Cocaine

Q: Sentence: No one would resist a pot of soup
A: Contains Code word:No,

Q: My cousin did the same and when the legalized pot in dc they really started cracking down in virginia and maryland.
A: Contains Code word: Yes, Code word : pot , Code word meaning : Marijuana

Q: i understand this is to get more customers but imo its bullshit
A: Contains Code word:No,

Q: i know whit doc got caught with 50 coke but is there tests of other vendors product
A:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: Contains Code word: Yes, Code word : coke , Code word meaning : Cocaine 


In [46]:
import pandas as pd
import re
from google.colab import auth as google_auth
google_auth.authenticate_user()

import vertexai
from vertexai.language_models import TextGenerationModel
import time

In [47]:
df =  pd.read_csv('CombinesDS_2.txt', delimiter='|')
dataset_size = 0.5
log_fname = "PaLM_test_log1.txt"
result_file = "PaLM_test_result1.txt"

vertexai.init(project="researchproposal", location="us-central1")
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 1
}
model = TextGenerationModel.from_pretrained("text-bison@001")

In [48]:
fewshot_msg="""You are an investigator looking for code words for drugs hidden in sentences. You have to analyze the sentences and determine whether they contain a code word for drugs. If the sentence contains a code word for drugs, answer as Present: yes and identify the code word in the sentence. If the sentence does not contain any code word for drugs then answer as Present: no.

Q: Lol, that shit is probably so stepped on you can\'t even call it coke anymore.
A: Present: Yes, Code word : coke, Code word meaning : Cocaine

Q: No one would resist a pot of soup
A: Present:No

Q: My cousin did the same and when the legalized pot in dc they really started cracking down in virginia and maryland.
A: Present: Yes, Code word : pot , Code word meaning : Marijuana

Q: i understand this is to get more customers but imo its bullshit
A: Present:No
"""

In [50]:
def format_msg(msg):
    msg = msg.rstrip().replace("'","")
    if msg[-1] != ".":
        msg += "."
    return msg

def call_palm(fewshot_msg):
    global model
    err_flag = False
    try:
        response = model.predict(fewshot_msg, **parameters)
        return response.text,err_flag
    except Exception as e :
        err_flag = True
        err_msg = f"API error {e}"
    if err_flag:
        return err_msg,err_flag

def logger(fname,msg):
    file= open(fname,'a',encoding='utf-8')
    file.write(f'{msg} \n')
    file.close()

def init_write(fname, text):
    file= open(fname,'w',encoding='utf-8')
    file.write(text + '\n')
    file.close()

def log(msg):
    global log_fname
    logger(log_fname, msg)


init_write(log_fname,"Logging starts")
init_write(result_file," ")


In [ ]:
task_msg= "Q: i know whit doc got caught with 50 coke but is there tests of other vendors product \nA: \n"
fewshot_msg = fewshot_msg + task_msg
fewshot_msg

In [51]:
counter = 0
prompt_msg = ""
exception_counter = 0
pattern_yes =  r'Present\s*:\s*yes*\s*\,\s*Code word\s*:\s*\w*\s*\w*\s*\,\s*Code word meaning\s*:\s*\w*\s*\w*\s*'
pattern_no = r'Present\s*:\s*no\s*'
for index,row in df.iterrows():
    if index == (int(len(df)*dataset_size)): # stop at 50% of dataset
    #if index == 4:  # stop at 50% of dataset
        break
    log (f" Sentence: {row['Sentences']}")
    counter += 1
    prompt_msg =  f"Q:{ format_msg(row['Sentences'])}\nA:\n"
    task_prompt = fewshot_msg + prompt_msg
    #log(task_prompt)
    result,err_flg = call_palm(task_prompt)
    log (result)
    if not err_flg:
        try:
            match_yes = re.fullmatch(pattern_yes,result,re.IGNORECASE)
            match_no = re.fullmatch(pattern_no,result,re.IGNORECASE)
            if match_yes or match_no:
                val = result.split(",")
                present = val[0].split(":")[1]
                if present.lower().strip() == "yes":
                    code_word = val[1].split(":")[1]
                    cw_meaning = val[2].split(":")[1]
                    logger(result_file,f"{row['Sentences']} | {present} | {code_word} | {cw_meaning}")
                else:
                    logger(result_file,f"{row['Sentences']} | {present} | NA | NA")
            else:
                log("Response pattern is not as expected")
                logger(result_file,f"{row['Sentences']} | Pattern not matching | NA | NA")
        except Exception as e:
            log(f"In exception due to {e}")
            exception_counter += 1
            logger(result_file,f"{row['Sentences']} | In exception {e} | NA | NA")
    else:
        exception_counter += 1
        log(f"In API exception{result}")
        logger(result_file,f"{row['Sentences']} | In API exception {result} | NA | NA")
    if exception_counter == 10:
        log("Encountered too many exceptions, going to exit")
        break
    time.sleep(5)

print("Done with execution")

Done with execution
